<a href="https://colab.research.google.com/github/otnemrasordep/DadaGP-track-processing/blob/main/bandGP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/bandGP


/content/drive/MyDrive/bandGP


In [4]:
!ls

filtered_dg-dg-b.zip  filtered_drums.zip  wandb


In [6]:
# Install necessary libraries
!pip install transformers datasets wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [7]:
# Import required libraries
import wandb
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset

In [8]:
# Initialize Weights & Biases
wandb.login(relogin=True)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:
# Start a new W&B run with increased timeout
wandb.init(
    project="bandGP-finetuning-t5",
    entity="otnemrasordep-queen-mary-university-of-london",
    settings=wandb.Settings(init_timeout=120),  # Increase timeout to 120 seconds
)

wandb: Currently logged in as: otnemrasordep (otnemrasordep-queen-mary-university-of-london). Use `wandb login --relogin` to force relogin


In [11]:
# Extract INPUT
import zipfile
import os
from tqdm.notebook import tqdm

# Path to your ZIP file
zip_file_path = "/content/drive/MyDrive/bandGP/filtered_dg-dg-b.zip"
# Destination folder for extracted files
extract_to = "/content/drive/MyDrive/bandGP/"

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Get the list of files in the ZIP archive
    file_list = zip_ref.namelist()
    total_files = len(file_list)

    # Use tqdm to show progress
    with tqdm(total=total_files, desc="Extracting files") as pbar:
        for file in file_list:
            zip_ref.extract(file, extract_to)
            pbar.update(1)  # Update progress


Extracting files:   0%|          | 0/49574 [00:00<?, ?it/s]

In [15]:
len(os.listdir('/content/drive/MyDrive/bandGP/filtered_dg-dg-b'))

49572

In [17]:
!rm -r /content/drive/MyDrive/bandGP/filtered_drums

In [ ]:
# Extract OUTPUT
import zipfile
import os
from tqdm.notebook import tqdm

# Path to your ZIP file
zip_file_path = "/content/drive/MyDrive/bandGP/filtered_drums.zip"
# Destination folder for extracted files
extract_to = "/content/drive/MyDrive/bandGP/"

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Get the list of files in the ZIP archive
    file_list = zip_ref.namelist()
    total_files = len(file_list)

    # Use tqdm to show progress
    with tqdm(total=total_files, desc="Extracting files") as pbar:
        for file in file_list:
            # print(file)
            zip_ref.extract(file, extract_to)
            pbar.update(1)  # Update progress

Extracting files:   0%|          | 0/49574 [00:00<?, ?it/s]

In [ ]:
len(os.listdir('/content/drive/MyDrive/bandGP/filtered_drums'))

In [ ]:
import json

# Collect input-output pairs
# T5 expects a "Input: xxx; Output: yyy" format for every file
pairs = []

# Input and output directories
input_dir = "/content/filtered_dg-dg-b"
output_dir = "/content/filtered_drums"

# Ensure files are sorted for pairing
input_files = sorted(os.listdir(input_dir))
output_files = sorted(os.listdir(output_dir))

# Create pairs
for x_file, y_file in zip(input_files, output_files):
    x_path = os.path.join(input_dir, x_file)
    y_path = os.path.join(output_dir, y_file)

    with open(x_path, "r") as x, open(y_path, "r") as y:
        input_text = x.read().strip()
        output_text = y.read().strip()
        pairs.append(f"Input: {input_text}\nOutput: {output_text}")

with open("paired-dataset-dgdgb-drums.json", "w") as final:
	json.dump(pairs, final)